In [ ]:
# requests will allow us to make a web request
import requests
# BeautifulSoup will allow us to easily parse the website's HTML code
from bs4 import BeautifulSoup
import re
import pandas as pd
import string
import numpy as np

pd.options.display.max_colwidth = 2000
pd.options.display.max_rows = 15000
pd.options.display.max_columns = 150

## Step 1. Fetching Team IDs from the Ranking Page

In [ ]:
%%time

# Using the ranking page to get the list of teams and their ids
url2 = "https://stats.ncaa.org/selection_rankings/nitty_gritties/27203"
header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
        }
source = requests.get(url2, headers=header).text
soup2 = BeautifulSoup(source, 'lxml')

In [ ]:
# print(soup2.prettify())

In [ ]:

teams_df = pd.DataFrame(columns=['Team', 'Team_ID'])
for team_id in soup2.find_all('a', class_="skipMask", target="TEAM_PAGE", href = True):
#     print(team_id.text, team_id['href'].rsplit('/', 1)[-1])
    teams_df = teams_df.append({'Team' : team_id.text, 'Team_ID' : team_id['href'].rsplit('/', 1)[-1]}, ignore_index=True)

teams_df

In [ ]:
teams_df

In [ ]:
teams_df.to_csv("teams_table.csv")

## Step 2. Use the Team ID to get the Game IDs

In [ ]:

games_df = pd.DataFrame(columns=['Team', 'Team_ID', 'Game_ID', 'Date', 'Opp_Team', 'Opp_Team_ID'])

for team_id in teams_df.Team_ID:
    print(team_id)
    
    url_team = "https://stats.ncaa.org/teams/"+str(team_id)+""
    header_team = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',}
    source = requests.get(url_team, headers=header_team).text
    soup_team = BeautifulSoup(source, 'lxml')
    
#     game_date     = soup_team.find('td', text=re.compile('Game Date:')).find_next('td').text.strip()
#     game_location = soup_team.find('td', text=re.compile('Location:')).find_next('td').text.strip()
#     away_team     = (soup_team.find('a', class_="skipMask", target="TEAMS_WIN")['href'].rsplit('/', 1)[-1])
#     home_team     = (soup_team.find_all('a', class_="skipMask", target="TEAMS_WIN")[-1]['href'].rsplit('/', 1)[-1])
    
    for game in soup_team.find_all('a', class_="skipMask", target="BOX_SCORE_WINDOW", href = True):
#         print(game['href'].rsplit('/')[-2])
        url_team2 = "https://stats.ncaa.org"+game['href']+""
        header_team2 = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',}
        source2 = requests.get(url_team2, headers=header_team2).text
        soup_team2 = BeautifulSoup(source2, 'lxml')
        
        for game2 in soup_team2.find_all('a', text='Play by Play', href = True):
            games_df = games_df.append({'Team' : teams_df.loc[teams_df['Team_ID'] == team_id, 'Team'].iloc[0], 
                                    'Team_ID' : team_id,
                                    'Game_ID' : game2['href'].rsplit('/')[-1]
                                   }, ignore_index=True)

games_df

In [ ]:
games_df

In [ ]:
games_df.to_csv("game_ids_table.csv")

## Step 3. Fetching Play-by-Play Details

In [ ]:
%%time

pbp_recs=[{}]


for game_id in games_df.Game_ID:
    print(game_id)
    
    url_game = "https://stats.ncaa.org/game/play_by_play/"+str(game_id)
    header_game = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',}
    source = requests.get(url_game, headers=header_game).text
    soup_game = BeautifulSoup(source, 'lxml')
    
    #print(soup_game.find_all('table', class_ = 'mytable', width = "1000px"))
    
    game_date     = soup_game.find('td', text=re.compile('Game Date:')).find_next('td').text.strip()
    game_location = soup_game.find('td', text=re.compile('Location:')).find_next('td').text.strip()
    away_team     = (soup_game.find('a', class_="skipMask", target="TEAMS_WIN")['href'].rsplit('/', 1)[-1])
    home_team     = (soup_game.find_all('a', class_="skipMask", target="TEAMS_WIN")[-1]['href'].rsplit('/', 1)[-1])
    
    i=1 # Set the inning counter

    # Loop through innings
    for inning in soup_game.find_all('table', class_ = 'mytable', width = "1000px"):
    # Loop through records in each inning, ignore tags with "grey_heading" class
        for tag in inning.find_all('tr', class_=lambda x: x != 'grey_heading')[2:]:
    # Check for the hyphen in the score, ignore "blank" rows without a score
            if tag.find(string=re.compile("-")):
                score = tag.select('td:nth-of-type(2)')[0].string
                
    # Determine if data is for the top or bottom of the inning
                if tag.select('td:nth-of-type(3)')[0].string is None:
                    pbp_recs.append({'Game_ID' : game_id, 'Date' : game_date, 'Location' : game_location, 'awayTeam' : away_team, 'homeTeam' : home_team, 'inn' : i, 'topBottom' : 'top', 'text' : tag.select('td:nth-of-type(1)')[0].string, 'awayScore' : score.split('-')[0].strip(), 'homeScore' : score.split('-')[1].strip()})
    #                print('Top ' + str(i) + '...' + tag.select('td:nth-of-type(1)')[0].string + ' ...Score ' + str(score))
                else:
                    pbp_recs.append({'Game_ID' : game_id, 'Date' : game_date, 'Location' : game_location, 'awayTeam' : away_team, 'homeTeam' : home_team, 'inn' : i, 'topBottom' : 'bot', 'text' : tag.select('td:nth-of-type(3)')[0].string, 'awayScore' : score.split('-')[0].strip(), 'homeScore' : score.split('-')[1].strip()})
    #                print('Bottom ' + str(i) + '...' + tag.select('td:nth-of-type(3)')[0].string + ' ...Score ' + str(score))
        
        i+=1
    # To create a dataframe: df = pd.DataFrame(pbp_recs)
    
pbp_df = pd.DataFrame(pbp_recs)

In [ ]:
pbp_df = pd.DataFrame(pbp_recs)
pbp_df

In [ ]:
final_pbp_df = pbp_df.drop([0]).reset_index(drop=True)
new_pbp_df = final_pbp_df

In [ ]:
final_pbp_df.head()

In [ ]:
final_pbp_df.to_csv("raw_pbp_data_table.csv")

In [ ]:
final_pbp_df = pd.read_csv('raw_pbp_data_table.csv')
final_pbp_df.head()
new_pbp_df = final_pbp_df

## Step 4. Extracting Pay by Play data from 'Text' column

In [ ]:
new_pbp_df = new_pbp_df['text'].str.split(";",expand = True)
new_pbp_df

In [ ]:



for col,data in new_pbp_df.iteritems():
    
    player_name_list = []
    action_list = []
    balls_list = []
    strikes_list = []
    
    for play in data.values:
        if play and pd.isna(play) == False:
#             print(play)

            player_name = ''
            action = ''
            balls = ''
            strikes = ''
            

            play = play.strip()

            player_name_search = re.match("(^[A-Z](\.|,)\s+[A-Za-z]+)|(^[A-Za-z]+,\s+[A-Z]\.)",play) # Example for name structure in the play text : A. Xyz OR Xyz, A. OR A, Xyz
            if player_name_search : player_name = player_name_search.group(0).strip()

            # searching for the action; extracting the balls and strikes
            action = (play.replace(player_name,"")).strip()
            score_status = re.search("\([0-9]\-[0-9](\s)*[A-Z]*\)\.$",action)

            if score_status:
                x = score_status.group(0)
                search_score = re.findall("[0-9]",x) # finding the balls and strikes in the text using regex
                balls = search_score[0]
                strikes = search_score[1]
                action = (action.replace(x,"")).strip()

#             print(player_name,action,balls,strikes)
            
            player_name_list.append(player_name)
            action_list.append(action)
            balls_list.append(balls)
            strikes_list.append(strikes)
        
        else:
            player_name_list.append("")
            action_list.append("")
            balls_list.append("")
            strikes_list.append("")
            
    final_pbp_df['player_name_'+str(col)] = pd.Series(player_name_list)
    final_pbp_df['action_'+str(col)] = pd.Series(action_list)
    final_pbp_df['balls_'+str(col)] = pd.Series(balls_list)
    final_pbp_df['strikes_'+str(col)] = pd.Series(strikes_list)
                
                
                

In [ ]:
final_pbp_df.head(200)

In [ ]:
# final_pbp_df[final_pbp_df['player_name_3'] != '']

In [ ]:
# final_pbp_df.to_excel("pbp_v1.xlsx")

## Step 5. Extracting details (runs, outs, error, earned run) from the text column

In [ ]:
pbp_details_df = final_pbp_df
pbp_details_df.head(50)

In [ ]:



# for data in pbp_details_df.iteritems():
    
runs_scored_list = []
outs_list = []
error_flag = []
run_credited_batter_flag = []

for play in pbp_details_df['text'].values:
    if play and pd.isna(play) == False:
#         print(play)

        play = play.strip()
        runs_scored =''
        outs = ''
        errors = 0
        runs_credited = ''


        # Runs Scored & Unearned or not to batter
        if re.search("([0-9]\sRBI)",play) :
            runs = re.search("([0-9]\sRBI)",play).group(0)
            runs_scored = int(runs.replace(" RBI",""))
            if re.search("(unearned)",play) :
                runs_credited = 0
            else :
                runs_credited = 1
        elif re.search("(RBI)",play) :
            runs_scored = 1
            if re.search("(unearned)",play) :
                runs_credited = 0
            else :
                runs_credited = 1



        # Outs
        if re.search("(into\sdouble\splay)",play) :
            outs = 2
        elif re.findall("out",play) :
            outs = len(re.findall("out",play))

        
        # Errors
        if re.search("error",play) :
#             print("found error")
            errors = 1
        



        runs_scored_list.append(runs_scored)
        outs_list.append(outs)
        error_flag.append(errors)
        run_credited_batter_flag.append(runs_credited)

    else:
        runs_scored_list.append("")
        outs_list.append("")
        error_flag.append("")
        run_credited_batter_flag.append("")

pbp_details_df['runs_scored'] = pd.Series(runs_scored_list)
pbp_details_df['outs'] = pd.Series(outs_list)
pbp_details_df['error_flag'] = pd.Series(error_flag)
pbp_details_df['run_credited_batter_flag'] = pd.Series(run_credited_batter_flag)

                
                

In [ ]:
pbp_details_df.head(500)

In [ ]:
# pbp_details_df[pbp_details_df['error_flag'] == 1]

In [ ]:
pbp_details_df.to_csv("text_separated_pbp_data.csv")

In [ ]:
STOP!

##### Box Score Code

In [ ]:
# This cell scrapes the simple "box score" for the game and puts it into a list of dictionaries (which...
# is easily converted into a pandas dataframe!)

game_inn_scores=[{}]

# Get the gameid...
gameid = int(soup.find('ul',class_ = "level1",id = "root").select("li > a")[0]['href'].rsplit('/', 1)[-1])

# Get the total number of innings in the game...
num_innings = len(soup.find('table', class_ = 'mytable', width = "40%").tr.find_all())-4

# Get the teamid of the AWAY team
away_team = int(soup.find('a', class_="skipMask", target="TEAMS_WIN")['href'].rsplit('/', 1)[-1])
# I'm definitely being a little sloppy here, but...
# soup.find('a', class_="skipMask", target="TEAMS_WIN")['href']
# ...is exactly equivalent to...
# temp = soup.find('a', class_="skipMask", target="TEAMS_WIN")
# temp['href']

# Get the teamid of the HOME team
home_team = int(soup.find_all('a', class_="skipMask", target="TEAMS_WIN")[-1]['href'].rsplit('/', 1)[-1])

# Get the scores for the AWAY team
away_inn_scores=soup.find('table', class_ = 'mytable', width = "40%").select('tr')[1].find_all('td')
# Get the scores for the HOME team
home_inn_scores=soup.find('table', class_ = 'mytable', width = "40%").select('tr')[2].find_all('td')

# Capture the game notes (if there are any--and they may be multiple)
# These may help us down the road to match pitchers with hitters in each at-bat
notes = ''
for tag in soup.find_all('table', align="center", width="50%")[0].find_all('td'):
    if not tag.text.startswith('Weather:') and not tag.text.startswith('Game:'):
        notes += tag.text + ";"

# Get the game date
game_date = soup.find('td', text=re.compile('Game Date:')).find_next('td').text.strip()

# Get the game location
game_location = soup.find('td', text=re.compile('Location:')).find_next('td').text.strip()

# Get rid of team name item
away_inn_scores.pop(0)
home_inn_scores.pop(0)

# Remove and store total errors for game
a_e = away_inn_scores.pop()
h_e = home_inn_scores.pop()

# Remove and store total hits for game
a_h = away_inn_scores.pop()
h_h = home_inn_scores.pop()

# Remove and store total runs for game
a_r = away_inn_scores.pop()
h_r = home_inn_scores.pop()

# Loop through innings and store values properly
for i in range(len(away_inn_scores)):
    game_inn_scores.append({'gameid' : gameid, 'teamid' : away_team, 'inn' : i + 1, 'runs_scored' : away_inn_scores[i].text})
    game_inn_scores.append({'gameid' : gameid, 'teamid' : home_team, 'inn' : i + 1, 'runs_scored' : home_inn_scores[i].text})

# To create a dataframe: df = pd.DataFrame(game_inn_scores)

In [ ]:
df_game_inn_scores = pd.DataFrame(game_inn_scores)
df_game_inn_scores